# Camera Motion Test -- Optical Flow

In [11]:
import cv2
import numpy as np
from pathlib import Path
import pandas as pd

Import video

In [12]:
video_number = "8"
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
video_path = project_root / "data" / f"recording_{video_number}" / f"Recording_{video_number}.mp4" 
video_path = str(video_path)

# Load the video
cap = cv2.VideoCapture(video_path)

# Check
print(f"Opened: {cap.isOpened()}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, Total Frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}")

Opened: True, FPS: 30.0, Total Frames: 119.0


Estimate the camera motion with optical flow

In [13]:
def estimate_background_motion(cap, max_frames=200):
    orb = cv2.ORB_create(nfeatures=1000)  # more features helps

    ret, prev_frame = cap.read()
    if not ret:
        return []

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_kp, prev_desc = orb.detectAndCompute(prev_gray, None)

    motions = []
    dxs = []
    dys = []

    for _ in range(max_frames):
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kp, desc = orb.detectAndCompute(gray, None)

        if prev_desc is not None and desc is not None:
            # Match ORB descriptors
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            matches = bf.match(prev_desc, desc)

            # Extract matched keypoints
            src_pts = np.float32([prev_kp[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

            # Use RANSAC to filter out moving objects
            if len(src_pts) >= 10:   # tune
                H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
                if H is not None:
                    # Extract translation components from homography
                    dx, dy = H[0, 2], H[1, 2]
                    motion_magnitude = np.sqrt(dx**2 + dy**2)
                    motions.append(motion_magnitude)
                    dxs.append(dx)
                    dys.append(dy)

        prev_gray = gray
        prev_kp, prev_desc = kp, desc

    cap.release()
    return motions, dxs, dys


In [14]:
motions, dx, dy = estimate_background_motion(cap, max_frames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
avg_motion = np.mean(motions)

if avg_motion < 1:
    print("Camera is still 🎥, motion:", avg_motion)
else:
    print("Camera is moving 📷, motion:", avg_motion)

Camera is still 🎥, motion: 0.962806187591208


Save the motions in a csv file

In [15]:
# output path
output_motions_path = project_root / "notebook" / "lane_detection" / "intermediate_data" / "background_motion" / f"motion_{video_number}.csv"
# save motions in a csv file
df = pd.DataFrame({'dx': dx, 'dy': dy})
df.to_csv(output_motions_path, index=False)

print(f"Motion data saved to {output_motions_path}")

Motion data saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\notebook\lane_detection\intermediate_data\background_motion\motion_8.csv
